In [ ]:
%pip install -U scikit-learn
%pip install gradio

In [ ]:
import tensorflow as tf
import os
import gradio as gr

gpus = tf.config.experimental.list_physical_devices('GPU')
gpus

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
import cv2
import imghdr

In [ ]:
data_dir = 'data'
test_dir = 'test_images'
classes = ['Bulging Eyes', 'Cataracts', 'Crossed Eyes', 'Glaucoma', 'Healthy Eyes', 'Uveitits']

In [ ]:
image_exts = ['jpeg', 'jpg', 'bmp', 'png']

In [ ]:
for image_class in os.listdir(data_dir):
    for image in os.listdir(os.path.join(data_dir, image_class)):
        image_path = os.path.join(data_dir, image_class, image)
        try:
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            if tip not in image_exts:
                print('Image not in ext list {}'.format(image_path))
                os.remove(image_path)
        except Exception as e:
            print('Issue with image {}'.format(image_path))

In [ ]:
for image in os.listdir(test_dir):
        try:
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            if tip not in image_exts:
                print('Image not in ext list {}'.format(image_path))
                os.remove(image_path)
        except Exception as e:
            print('Issue with image {}'.format(image_path))

In [ ]:
tf.data.Dataset

import numpy as np
from matplotlib import pyplot as plt



In [ ]:
data = tf.keras.utils.image_dataset_from_directory('data')
#tests = tf.keras.utils.image_dataset_from_directory('test_images')

In [ ]:
data_iterator = data.as_numpy_iterator()

In [ ]:
# Get another batch from the iterator 
batch = data_iterator.next()

In [ ]:
# images Represented as numpy arrays
batch[0].shape

In [ ]:
# Class 1 = Sad 
# Class 0 = happy
batch[1]

In [ ]:
fig, ax = plt.subplots(ncols = 5, figsize =(20,20))
for idx, img in enumerate(batch[0][:5]):
    ax[idx].imshow(img.astype(int))
    ax[idx].title.set_text(batch[1][idx])

In [ ]:
scaled = batch[0] / 255

In [ ]:
data = data.map(lambda x, y: (x / 255, y))

In [ ]:
scaled_iterator = data.as_numpy_iterator()

In [ ]:
batch = scaled_iterator.next()

In [ ]:
batch[0].max()

In [ ]:
fig, ax = plt.subplots(ncols = 5, figsize =(20,20))
for idx, img in enumerate(batch[0][:5]):
    ax[idx].imshow(img)
    ax[idx].title.set_text(batch[1][idx])

In [ ]:
len(data)


In [ ]:
train_size = int(len(data) * .7)
val_size = int(len(data) * .2)
test_size = int(len(data) * .1) + 2

train_size 

In [ ]:
train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size+val_size).take(test_size)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, BatchNormalization, Dropout

model = Sequential()


model.add(Conv2D(64, (3,3), 2, padding = 'same', activation = 'relu', input_shape = (256, 256, 3)))
model.add(MaxPooling2D(pool_size = 2, strides = 2))
model.add(Conv2D(64, (3,3), 2, padding = 'same', activation = 'relu', ))
model.add(MaxPooling2D(pool_size = 2, strides = 2))
model.add(Conv2D(128, (3,3), 2, padding = 'same', activation = 'relu', ))
model.add(Conv2D(128, (3,3), 2, padding = 'same', activation = 'relu', ))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = 2, strides = 2))
model.add(MaxPooling2D(pool_size = 2, strides = 2))
model.add(Flatten())
model.add(Dense(512, activation = 'relu'))
#model.add(Dropout(.2))
model.add(Dense(512, activation = 'relu'))
#model.add(Dropout(.2))
model.add(Dense(6, activation = 'softmax'))


model.compile('adam', loss = tf.losses.sparse_categorical_crossentropy, metrics =['accuracy'])

model.summary()

logdir = 'logs'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logdir)
hist = model.fit(train, epochs = 100, validation_data = val)

In [ ]:
fig = plt.figure()
plt.plot(hist.history['loss'], color = 'teal', label = 'loss')
plt.plot(hist.history['val_loss'], color = 'orange', label = 'val_loss')
fig.suptitle('Loss', fontsize = 20)
plt.legend(loc = "upper left")
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(hist.history['accuracy'], color = 'teal', label = 'accuracy')
plt.plot(hist.history['val_accuracy'], color = 'orange', label = 'val_accuracy')
fig.suptitle('Accuracy', fontsize = 20)
plt.legend(loc = "upper left")
plt.show()

In [ ]:
#You can test the NN here by passing in an image
img = cv2.imread('/Users/jbullock/EyeCU/test_images/cataracttest.jpg')
#/Users/jbullock/EyeCU/test_images/uveitistest2.jpg
#C:/Users/Ethan/OneDrive/Documents/GitHub/EyeCU/test_images/uveitistest2.jpg
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
#Have to resize the image before we pass it into the NN
resize = tf.image.resize(img, (256, 256))
img_array = tf.keras.utils.img_to_array(resize)
img_array = tf.expand_dims(resize, 0) # Create a batch

plt.imshow(resize.numpy().astype(int))
plt.show()

In [ ]:

preds = np.round(model.predict(batch[0]), 1)
#r = np.round(preds,5).argmax()
#r
preds
for i in range(6):
    for j in range(6):
        if preds[i][j] == 1:
            print('This image belongs to: ' + classes[j])
            
    

In [ ]:
pred = np.round(preds,6).argmax()


print(preds[0])

pred_labels = []

for i in range(6):
    r = np.round(preds[i],0).argmax()
    print(r)
    if r ==0 : pred_labels.append("Bulging Eyes")
    elif r ==1: pred_labels.append("Cataracts")
    elif r ==2: pred_labels.append("Crossed Eyes")
    elif r ==3: pred_labels.append("Glaucoma")
    elif r ==4: pred_labels.append("Healthy Eyes")
    elif r ==5: pred_labels.append("Uveitis")

In [ ]:
images = batch[0][:6]

print(images.shape)

fig = plt.figure(figsize=(20, 20))
for m in range(1, 7):
    img = images[m-1].reshape([256, 256, 3])
    fig.add_subplot(5, 5, m)
    plt.imshow(img)
    plt.title("Pred: " + pred_labels[m-1])
    plt.axis('off')
plt.show()

In [ ]:

img_array = tf.keras.utils.img_to_array(resize)
img_array = tf.expand_dims(resize, 0) # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions)

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(classes[np.argmax(score)], 100 * np.max(score))
)
np.argmax(score)
score

In [ ]:

def predict_image(img):
  img_array2 = tf.keras.utils.img_to_array(img)
  img_array2 = tf.expand_dims(img, 0)
  prediction = np.round(model.predict(img_array2))
  for i in range(6):
    for j in range(6):
      if prediction[i][j] == 1:
        return classes[j]


In [60]:
image = gr.inputs.Image(shape=(256,256))
label = gr.outputs.Label(num_top_classes=6)

#interface = gr.Interface.load("huggingface/jazmineox/Eye_classification", fn=predict_image, inputs=image, outputs=label,interpretation='default')
gr.Interface(fn=predict_image, inputs=image, outputs=label,interpretation='default').launch(debug='True', share=True)

#interface.launch()

Keyboard interruption in main thread... closing server.


In [ ]:
model.save("model.hdf5")